<h3>0. Exploratory analysis</h3>

In [3]:
import sys


In [466]:
import json
import pandas as pd

df_training=pd.read_json('project_files/training.json', encoding = 'utf8')
df_devel=pd.read_json('project_files/devel.json')
df_docs=pd.read_json('project_files/documents.json')
df_testing=pd.read_json('project_files/testing.json')

df_training=pd.read_pickle('project_files/df_training.pkl')
df_training=df_training.iloc[0:2000]
#df_docs=df_docs.iloc[0:10]

In [464]:
df_training.head()

,answer_paragraph,docid,question,text,NE_question,NE_text
0,23,0,A kilogram could be definined as having a Planck constant of what value?,6966662606895999999♠6.62606896×10−34 j⋅s,[],"[(6966662606895999999 ♠ 6.62606896, NUMBER), (10 − 34, NUMBER)]"
1,22,0,What is the shape of the object that establishes the base unit of the kilogram?,cylinder,[],[]
2,12,0,What example is given as another paired relationship of uncertainly related to standard deviation?,time vs. energy,[],[]
3,1,0,What does the Planck Constant refer to?,quantum of action,[],[]
4,10,0,When was the first quantized model of the atom introduced?,1913,"[(first, ORDINAL), (model, TITLE)]","[(1913, DATE)]"


<h3>1. Question processing</h3>

1. Find Keywords
2. Answer types - Using answer type taxonomy
3. Query formulation -> Keywords
4. Go to each document and check the frequency distribution of words and pick the document if one of the query words are present in document. Create a rank with that score
5. Find the paragraphs -> Discard irrelevant paragraphs. Use NE,Keywords, longest exact keywords. Put same weight for now and calculate the score of paragraphs. Rank each of the paragraphs in the document. We have to use the original answer and match the answer type
6. Find candidate answers -> Use supervised ML method
7. Merge candidate answers -> Use NER
8. Pick the best answer -> Logistic regression

Configuring Stanford CoreNLP . Link -> https://blog.manash.me/configuring-stanford-parser-and-stanford-ner-tagger-with-nltk-in-python-on-windows-f685483c374a

In [436]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tag.stanford import CoreNLPNERTagger
from itertools import groupby

stopwords = set(nltk.corpus.stopwords.words('english')) 
stemmer = nltk.stem.PorterStemmer() 

def get_Name_Entity_NLTK(data):
    results=[]
    for sentence in data:
        ne_chunked_sents = ne_chunk(pos_tag(word_tokenize(sentence)))
        result = []

        for tagged_tree in ne_chunked_sents:

            if hasattr(tagged_tree, 'label'):
                entity_name = ' '.join(c[0] for c in tagged_tree.leaves()) #
                entity_type = tagged_tree.label() # get NE category
                result.append((entity_name, entity_type))
        results.append(result)

    return results

def get_Name_Entity_Sentence(sentence):
    st = CoreNLPNERTagger(url='http://localhost:9000')
    tokenized_text = nltk.word_tokenize(sentence)
    classified_text = st.tag(tokenized_text)
    result = {}
    
    for res in classified_text:
        if len(res) > 0:
            for tag, chunk in groupby(classified_text, lambda x:x[1]):
               if tag != "O":
                    word = " ".join(w for w, t in chunk)
                    result[word.lower()] = tag
    
    
    return result

def get_Name_Entity_paragraph(paragraph):
    result = []
    tokenized_sentence = nltk.sent_tokenize(paragraph)
    for sentence in tokenized_sentence:
        result.append(get_Name_Entity_Sentence(sentence))
    
    return result

def get_Name_Entity_StanfordCoreNLP(data):
    st = CoreNLPNERTagger(url='http://localhost:9000')
    results=[]
    for sentence in data:
        tokenized_text = nltk.word_tokenize(sentence)
        classified_text = st.tag(tokenized_text)
        result = []
        for tag, chunk in groupby(classified_text, lambda x:x[1]):
            if tag != "O":
                word = " ".join(w for w, t in chunk)
                result.append((word.lower(),tag))
       
        results.append(result)
        
    return results

def addNameEntity(df,feature,func):
    if 'NE'+"_"+feature in df:
        df = df.drop('NE'+"_"+feature, axis=1)
    df["NE"+"_"+feature] = func(df[feature])
    
    return df

In [437]:
#df_training = addNameEntity(df_training,"question",get_Name_Entity_StanfordCoreNLP)
#df_training = addNameEntity(df_training,"text",get_Name_Entity_StanfordCoreNLP)

Get Keywords

In [438]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
POS = set(["NN","NNS","NNP","NNPS","CD","JJ","VB","VBD","VBG","VBN","VBP","VBZ"]) 

stopwords = set(nltk.corpus.stopwords.words('english')) 
stemmer = nltk.stem.PorterStemmer() 

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,wn.NOUN)
    if (lemma == word):
        lemma = lemmatizer.lemmatize(word,wn.VERB)
        
    return lemma

def get_keyword(data):
    result = []
    sentence=data
    tokenized_text = tokenizer.tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    for text,pos in tagged:
        text = lemmatize(text.lower())
        if text not in stopwords:
            if pos in POS:
                result.append(text)
                
    return result

def get_keyword_paragraph(data):
    results=[]
    tokenized_sentence = nltk.sent_tokenize(data)
    for sentence in tokenized_sentence:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def get_keyword_all(data):
    results=[]
    for sentence in data:
        result = get_keyword(sentence)
        results.append(result)
        
    return results

def add_keywords(df,feature):
    if 'keywords'+"_"+feature in df:
        df = df.drop('keywords'+"_"+feature, axis=1)
    df['keywords'+"_"+feature]=get_keyword_all(df[feature])
    return df

In [439]:
#df_training = add_keywords(df_training,'question')

<h4>Train a classifier</h4>

In [467]:
import nltk
def get_answer_features(passage,answer):
    
    sents_passage = nltk.sent_tokenize(passage)
    answer_sentence_ner={'UNKNOWN':'UNKNOWN'}
    answer_found='UNKNOWN'
    answer_sentence_keywords=[]
    common_entities=tuple()
    for sentence in sents_passage:
        if answer.lower() in sentence.lower():
            answer_found=sentence
            dict_answer_sentence_ner=get_Name_Entity_Sentence(sentence)
            dict_answer_ner=get_Name_Entity_Sentence(answer)
            common_entities = set(dict_answer_sentence_ner.items()) & set(dict_answer_ner.items())
            break
    
    return answer_found,common_entities

In [469]:
from collections import defaultdict

# get the training data set as we need it
answer_type={}
def prepare_training_data():
    
    df_training['answer_type']='UNKNOWN'
    df_training['answer_found']='UNKNOWN'
    df_training['common_entities']='UNKNOWN'
    df_training['NE_passage']='UNKNOWN'
    for index, row in df_training.iterrows():
        question=row['question']
        raw_answer=row['text']
        
        passage=df_docs.iloc[row['docid']]['text'][row['answer_paragraph']]
        
        answer_found,common_entities=get_answer_features(passage,raw_answer)
        
        if (len(common_entities)>0):
            ans=list(common_entities)[0][1]
            
        else:
            ans='UNKNOWN'
            
        df_training.at[index,'answer_type']=ans
        df_training.at[index,'answer_found']=answer_found
        df_training.at[index,'common_entities']=shared_items
        
        ###
        df_training.at[index,'NE_passage']=get_Name_Entity_paragraph(passage)
    
        
        

In [ ]:
prepare_training_data()

In [443]:
pd.set_option('display.max_colwidth', -1)
df_training[['question','text','answer_found','NE_passage','answer_type','NE_text','NE_question','common_entities']][0:20]

In [444]:
# BOW extraction for passages and questions


def get_passages_bow(passages):
    passage_bow={}
    for passage in passages:
        for token in nltk.word_tokenize(passage):
            if token not in stopwords: 
                word=stemmer.stem(token.lower())
                passage_bow[word] = passage_bow.get(word, 0) +  1
    
    return passage_bow

def get_sentences_bow(sentences):
    sentence_bow={}
    
    for sentence in sentences:
        for token in nltk.word_tokenize(sentence):
            if token not in stopwords:
                word=stemmer.stem(token.lower())
                sentence_bow[word] = sentence_bow.get(word, 0) +  1
    
    return sentence_bow


def get_question_bow(question,keywords):
    question_bow={}
    for token in nltk.word_tokenize(question):
        if token not in stopwords: 
            word=stemmer.stem(token.lower())
            if word in keywords:
                question_bow[word] = question_bow.get(word, 0) +  1
                
    return question_bow

In [454]:
def filter_questions():
    questions=[]
    questions_not_found=[]
    answers_not_found=[]
    classes=[]
    for index, row in df_training.iterrows():
        if len(row['common_entities'])>0 and row['answer_type']!='UNKNOWN':
            questions.append(row['question'])
            classes.append(row['answer_type'])
        else:
            questions_not_found.append(row['question'])
            answers_not_found.append(row['text'])
            
    return questions,classes,questions_not_found,answers_not_found
        
def get_feature_questions(questions, keywords):
    qs = []
    for question in questions:
        q_bow = get_question_bow(question,keywords)
        qs.append(q_bow)
        
    return qs

In [458]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer


    
# get the most common words from answer sentences (we can twek this for paragraph)
answer_sentences_bow=get_sentences_bow(df_training['answer_found'])
answer_keywords = set([word for word, count in answer_sentences_bow.items()])

#filter questions with common entities
questions,classes,questions_not_found,answers_not_found=filter_questions()


qs=get_feature_questions(questions,answer_keywords)


qs_training=qs
qs_testing=get_feature_questions(questions_not_found,answer_keywords)




    
    
   

In [459]:
qs_testing[0:20]

[{'base': 1,
  'establish': 1,
  'kilogram': 1,
  'object': 1,
  'shape': 1,
  'unit': 1},
 {'anoth': 1,
  'exampl': 1,
  'given': 1,
  'pair': 1,
  'relat': 1,
  'relationship': 1,
  'standard': 1},
 {'constant': 1, 'planck': 1, 'refer': 1},
 {'black': 1, 'bodi': 1, 'first': 1, 'radiat': 1, 'scientist': 1, 'studi': 1},
 {'1926': 1,
  'correct': 1,
  'electron': 1,
  'give': 1,
  'help': 1,
  'quantiz': 1,
  'rule': 1},
 {'classic': 1, 'exist': 1, 'mechan': 1, 'requir': 1, 'statist': 1},
 {'classic': 1, 'exist': 1, 'mechan': 1, 'requir': 1, 'statist': 1},
 {'contain': 1,
  'energi': 1,
  'eye': 1,
  'human': 1,
  'light': 1,
  'much': 1,
  'sensit': 1},
 {'also': 1, 'frequenc': 1, 'known': 1, 'scienc': 1},
 {'effect': 1, 'first': 1, 'observ': 1, 'photoelectr': 1},
 {"''": 1,
  ',': 1,
  '``': 1,
  'came': 1,
  'catastroph': 1,
  'term': 1,
  'ultraviolet': 1},
 {'act': 1, 'photoelectron': 1, 'term': 1, 'time': 1, 'use': 1, 'virtual': 1},
 {'atom': 1, 'nuclei': 1, 'properti': 1, 'unit':

In [462]:
if (len(qs)>0 and len(classes)>0):
    # fit vectorizer
    vectorizer = DictVectorizer()
    
    
    
    X_train_dtm = vectorizer.fit_transform(qs_training)
    
    X_test_dtm = vectorizer.transform(qs_testing[0:20])
    
    # tag the answers
    # fit a logistic regression model to the data 
    # build classifier
    model = MultinomialNB(2, False, None)

    # train the model using X_train_dtm 
    model.fit(X_train_dtm, classes)
    
    
    
    y_predicted_class = model.predict(X_train_dtm)
    
    #for i in range(20):
     #   print(questions_not_found[i]),
     #   print(answers_not_found[i]),
     #   print(y_predicted_class[i])
    
    check_results(y_predicted_class,classes)

Accuracy:
0.9453125
                   precision    recall  f1-score   support

   CAUSE_OF_DEATH       1.00      0.88      0.94        17
             CITY       0.00      0.00      0.00         1
          COUNTRY       0.90      0.95      0.92        37
  CRIMINAL_CHARGE       1.00      1.00      1.00         1
             DATE       0.92      0.99      0.96       216
         DURATION       0.90      0.95      0.92        19
         IDEOLOGY       1.00      0.67      0.80         3
         LOCATION       1.00      1.00      1.00         1
            MONEY       1.00      0.60      0.75         5
      NATIONALITY       1.00      0.71      0.83        24
           NUMBER       0.97      0.97      0.97       181
          ORDINAL       1.00      0.86      0.92         7
          PERCENT       1.00      1.00      1.00         9
           PERSON       0.93      0.95      0.94        57
         RELIGION       1.00      1.00      1.00        10
              SET       1.00      0

/Users/danielgil/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [430]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def check_results(predictions, classifications):
    print("Accuracy:")
    print(accuracy_score(classifications,predictions))
    print(classification_report(classifications,predictions))

In [ ]:
df_training

<h3>2. Candidate answering generation</h3>

<h4> Get a score for the passage to filter the most relevant passages</h4>


In [93]:
## features relevant to this part
# number of named entities of the right type in the passage
# number of question keywords in the passage
# the longest exact sequence of question keywords
# rank of the document where the passage was extracted
# proximity of the keywords from the original query
# ngram overlap between the passage and the question

First, we will set up useful functions to extract term frequencies to build the vector space model

In [294]:
import nltk
from collections import defaultdict
from collections import Counter
from math import log

stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

# get the terms for a passage
def get_terms(passage):
    terms = set()
    for token in nltk.word_tokenize(passage):
        if token not in stopwords: 
            terms.add(stemmer.stem(token.lower()))
    return terms
    
# get document_term 
def get_document_term_passsages(ds_documents):
    document_term={}
    passageID=0
    for index, row in ds_documents.iterrows():
        # every row is a document
        list_of_passages=row['text']
        for passage in list_of_passages:
            terms=get_terms(passage)
            document_term[passageID]=terms
            passageID+=1
        
    return document_term

# get the term frequency
def extract_term_freqs(doc):
    tfs = Counter()
    for token in doc:
        if token not in stopwords: 
            tfs[stemmer.stem(token.lower())] += 1
    return tfs
        
# compute idf
def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += 1
    return dfs
    

In [99]:
# create a document-term matrix
docs=get_document_term_passsages(df_docs)

In [100]:
# create a vector space model we need to define a score function
# first I will use tf-idf
doc_term_freqs = {}
for docid, terms in docs.items():
    term_freqs = extract_term_freqs(terms)
    doc_term_freqs[docid] = term_freqs

M = len(doc_term_freqs)

doc_freqs = compute_doc_freqs(doc_term_freqs)

<b>Improvement:</b> Use BM25

Create an inverted index for query processing. Inverted index will not change from query to query. Here we can improve how the weight is defined for the posting list tuple for each term (docid,weight)

In [101]:
## Code from WSTA_N16_information_retrieval
vsm_inverted_index = defaultdict(list)
for docid, term_freqs in doc_term_freqs.items():
    N = sum(term_freqs.values())
    length = 0
    
    # find tf*idf values and accumulate sum of squares 
    tfidf_values = []
    for term, count in term_freqs.items():
        tfidf = float(count) / N * log(M / float(doc_freqs[term]))
        tfidf_values.append((term, tfidf))
        length += tfidf ** 2

    # normalise documents by length and insert into index
    length = length ** 0.5
    for term, tfidf in tfidf_values:
        # note the inversion of the indexing, to be term -> (doc_id, score)
        vsm_inverted_index[term].append([docid, tfidf / length])
        
# ensure posting lists are in sorted order (less important here cf above)
for term, docids in vsm_inverted_index.items():
    docids.sort()


Query the VSM creating a score for each document (passage) and returning the top k

In [102]:
def query_vsm(query, index, k=10):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)

## end copied code

Test the query:

In [103]:
results = query_vsm([stemmer.stem(term.lower()) for term in "First recognized in 1900 by Max Planck".split()], vsm_inverted_index)
results

[(0, 0.680116589444509),
 (1, 0.30340274351915997),
 (6, 0.24364584220578492),
 (65, 0.17637606987370624),
 (105, 0.16152465265290256),
 (10, 0.14946948590599873),
 (3, 0.14114017042707586),
 (14, 0.13829184288746416),
 (11, 0.13227012047661246),
 (19, 0.10701743930986322)]

<h3>3. Candidate answering scoring</h3>

<h3>4. Answer and confidence</h3>